<a href="https://colab.research.google.com/github/ngupta23/pycaret_faqs/blob/main/time_series/pycaret_ts_custom_model_sktime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install & Import Library

In [1]:
try:
  import pycaret
except:
  !pip install pycaret-ts-alpha

In [2]:
import numpy as np
from pycaret.internal.pycaret_experiment import TimeSeriesExperiment
from pycaret.datasets import get_data

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [3]:
y = get_data("airline", verbose=False)

In [4]:
#### Simulate missing values ----
y[60:72] = np.nan

## Define your **custom** pipeline!

In [5]:
#### Create a sktime pipeline with preprocessing ----
from sktime.transformations.series.impute import Imputer
from sktime.transformations.series.boxcox import LogTransformer
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.forecasting.arima import ARIMA

my_forecaster = TransformedTargetForecaster(
    [
        ("impute", Imputer()),
        ("log", LogTransformer()),
        ("model", ARIMA(seasonal_order=(0,1,0,12))),
    ]
)

## Model with **custom** pipeline

In [6]:
exp = TimeSeriesExperiment()
exp.setup(data=y, fh=12, session_id=42)

,Description,Value
0,session_id,42
1,Original Data,"(144, 1)"
2,Missing Values,True
3,Transformed Train Set,"(120,)"
4,Transformed Test Set,"(12,)"
5,Fold Generator,ExpandingWindowSplitter
6,Fold Number,3
7,Enforce Prediction Interval,False
8,Seasonal Period Tested,12
9,Seasonality Detected,False


In [7]:
my_custom_model = exp.create_model(my_forecaster)

,cutoff,MAE,RMSE,MAPE,SMAPE,R2
0,1956-12,11.8065,12.6168,0.0329,0.0323,0.9482
1,1957-12,43.1813,45.1464,0.1146,0.1078,0.4660
2,1958-12,10.4920,12.2277,0.0253,0.0256,0.9666
Mean,NaN,21.8266,23.3303,0.0576,0.0553,0.7936
SD,NaN,15.1096,15.4271,0.0404,0.0373,0.2317


In [8]:
exp.plot_model(my_custom_model)

In [9]:
my_custom_model

TransformedTargetForecaster(steps=[('impute',
                                    Imputer(forecaster=None, method='drift',
                                            missing_values=None,
                                            random_state=None, value=None)),
                                   ('log', LogTransformer()),
                                   ('model',
                                    ARIMA(maxiter=50, method='lbfgs',
                                          order=(1, 0, 0), out_of_sample_size=0,
                                          scoring='mse', scoring_args=None,
                                          seasonal_order=(0, 1, 0, 12),
                                          start_params=None,
                                          suppress_warnings=False, trend=None,
                                          with_intercept=True))])